In [280]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [281]:
#加载tag表数据
train_tag = pd.read_csv('train_data/train_tag.csv')
test_tag = pd.read_csv('test_data/test_tag.csv')

In [282]:
#将训练数据和测试数据连接方便做特征
df_tag=pd.concat([train_tag,test_tag],axis=0,ignore_index=True)
df_tag['flag'] = df_tag['flag'].fillna(-1)

In [284]:
df_tag.isnull().sum()#观察缺失数据

id                                    0
flag                               4000
gdr_cd                                0
age                                   0
mrg_situ_cd                           0
acdm_deg_cd                           1
job_year                              0
ic_ind                                0
fr_or_sh_ind                          0
dnl_mbl_bnk_ind                       0
dnl_bind_cmb_lif_ind                  0
hav_car_grp_ind                       0
hav_hou_grp_ind                       0
l6mon_agn_ind                         0
frs_agn_dt_cnt                        0
vld_rsk_ases_ind                      0
fin_rsk_ases_grd_cd                   0
confirm_rsk_ases_lvl_typ_cd           0
cust_inv_rsk_endu_lvl_cd              0
l6mon_daim_aum_cd                     0
tot_ast_lvl_cd                        0
pot_ast_lvl_cd                        0
bk1_cur_year_mon_avg_agn_amt_cd       0
l12mon_buy_fin_mng_whl_tms            0
l12_mon_fnd_buy_whl_tms               0


In [283]:
df_tag = df_tag.drop(['edu_deg_cd','deg_cd','atdd_type'],axis=1)#三个特征缺失值较多，删掉

In [286]:
#观察数据大致分布
for feature in list(df_tag):
    print(feature + "的特征分布如下：")
    print("{}特征有个{}不同的值".format(feature,df_tag[feature].nunique()))
    print(df_tag[feature].value_counts())

id的特征分布如下：
id特征有个43923不同的值
UE789AD    1
U4E85FA    1
U55EEC3    1
U17BBC5    1
U070183    1
          ..
UC457B0    1
U1D83D0    1
UF59F9F    1
UEBAF23    1
U1329D6    1
Name: id, Length: 43923, dtype: int64
flag的特征分布如下：
flag特征有个3不同的值
 0.0    30970
 1.0     8953
-1.0     4000
Name: flag, dtype: int64
gdr_cd的特征分布如下：
gdr_cd特征有个3不同的值
M     25282
F     17819
\N      822
Name: gdr_cd, dtype: int64
age的特征分布如下：
age特征有个60不同的值
30    2776
31    2715
32    2479
33    2449
29    2336
28    2212
34    2112
27    2066
26    1826
25    1696
35    1686
36    1599
38    1520
37    1448
24    1317
39    1280
41    1039
40    1023
23     926
42     879
22     742
43     705
44     665
45     599
46     593
47     531
49     518
48     515
50     491
21     479
52     367
51     363
20     256
57     230
53     221
54     214
56     204
55     178
58     151
60      80
59      74
62      58
61      57
63      54
65      32
66      29
64      29
19      20
67      18
68      16
69      15
70      14
71    

Name: cur_debit_min_opn_dt_cnt, Length: 5826, dtype: int64
cur_credit_min_opn_dt_cnt的特征分布如下：
cur_credit_min_opn_dt_cnt特征有个2595不同的值
-1       2892
 4595     150
 4260     123
 3955     116
 4199     112
         ... 
 1485       1
 1702       1
 1364       1
 1492       1
 1254       1
Name: cur_credit_min_opn_dt_cnt, Length: 2595, dtype: int64
cur_debit_crd_lvl的特征分布如下：
cur_debit_crd_lvl特征有个6不同的值
10    23257
0     14062
20     5790
40      776
80       21
60       17
Name: cur_debit_crd_lvl, dtype: int64


In [287]:
#将类别特征转化编码
str_col = ['gdr_cd','mrg_situ_cd','acdm_deg_cd']
for col in str_col:
    df_tag[col] = pd.factorize(df_tag[col])[0]

In [288]:
#填充数值特征，根据数值分布筛选出部分特征用0填充'\N',其余使用'-1'填充
col_0 = ['job_year','frs_agn_dt_cnt','fin_rsk_ases_grd_cd','confirm_rsk_ases_lvl_typ_cd',
         'tot_ast_lvl_cd','pot_ast_lvl_cd','hld_crd_card_grd_cd','l1y_crd_card_csm_amt_dlm_cd']
df_tag[col_0].replace('\\N',0,inplace=True)
df_tag.replace('\\N',-1,inplace=True)

In [289]:
#信用卡：持卡天数*等级
df_tag['credit_level1']=df_tag['cur_credit_min_opn_dt_cnt']*pd.to_numeric(df_tag['l1y_crd_card_csm_amt_dlm_cd'])
df_tag['credit_level2']=df_tag['cur_credit_min_opn_dt_cnt']*pd.to_numeric(df_tag['perm_crd_lmt_cd'])
df_tag['credit_level3']=df_tag['cur_credit_min_opn_dt_cnt']*pd.to_numeric(df_tag['hld_crd_card_grd_cd'])
df_tag['level_level']=pd.to_numeric(df_tag['l1y_crd_card_csm_amt_dlm_cd'])*pd.to_numeric(df_tag['perm_crd_lmt_cd'])  #等级*等级
df_tag['credit_amount']=df_tag['cur_credit_min_opn_dt_cnt']*df_tag['cur_credit_cnt']    #持卡天数*持卡数量
#信用卡：持卡数量*等级
df_tag['amount_level1']=df_tag['cur_credit_cnt']*pd.to_numeric(df_tag['perm_crd_lmt_cd'])
df_tag['amount_level2']=df_tag['cur_credit_cnt']*pd.to_numeric(df_tag['l1y_crd_card_csm_amt_dlm_cd'])
df_tag['amount_level3']=df_tag['cur_credit_cnt']*pd.to_numeric(df_tag['hld_crd_card_grd_cd'])

df_tag1 = df_tag.copy()
df_tag1.shape

In [292]:
#加载交易表数据
train_trd = pd.read_csv('train_data/train_trd.csv')
test_trd = pd.read_csv('test_data/test_trd.csv')

In [293]:
#将训练数据和测试数据连接方便做特征
df_trd = pd.concat([train_trd,test_trd],axis=0,ignore_index=True)
df_trd['flag'] = df_trd['flag'].fillna(-1)

In [294]:
#转化时间，提取月份和日期
from datetime import datetime
df_trd['trx_tm'] = pd.to_datetime(df_trd['trx_tm'])
df_trd['trx_month'] = df_trd['trx_tm'].dt.month
df_trd['trx_day'] = df_trd['trx_tm'].apply(lambda x:x.strftime('%Y-%m-%d'))
df_trd['last_trx_tm'] = df_trd.groupby('id')['trx_tm'].transform('max')#提取最后一次交易时间
df_trd['last_day'] = df_trd.groupby('id')['trx_day'].transform('max')#提取最后一次交易日期
print('df_trd is done')

df_trd is done


In [297]:
df_trd.columns

Index(['id', 'flag', 'Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd',
       'Trx_Cod2_Cd', 'trx_tm', 'cny_trx_amt', 'trx_month', 'trx_day',
       'last_trx_tm', 'last_day'],
      dtype='object')

In [298]:
#t1统计交易余额，交易次数，最小交易额，最大交易额，交易额标准差，交易平均值
#t2统计交易天数,t3计算最后一次交易额
#t4统计最后一天交易余额，交易次数，最小交易额，最大交易额，交易额标准差，交易平均值
t0 = df_trd[['id','flag']].drop_duplicates().reset_index(drop=True)
t1 = df_trd.groupby('id',as_index=False)['cny_trx_amt'].agg(
    {'trx_sum':'sum','trx_count':'count','trx_min':'min','trx_max':'max','trx_std':'std','trx_mean':'mean'})
t2 = df_trd.groupby('id',as_index=False)['trx_day'].agg({'trx_days':'nunique'})
t3 = df_trd[df_trd['trx_tm']==df_trd['last_trx_tm']].groupby('id',as_index=False)['cny_trx_amt'].agg({'last_amt':'sum'})
t4 = df_trd[df_trd['trx_day']==df_trd['last_day']].groupby('id',as_index=False)['cny_trx_amt'].agg(
    {'last_day_sum':'sum','last_day_count':'count','last_day_min':'min',
     'last_day_max':'max','last_day_mean':'mean','last_day_std':'std'})

merge1 = pd.merge(t0,t1,on='id',how='left')
merge2 = pd.merge(merge1,t2,on='id',how='left')
merge3 = pd.merge(merge2,t3,on='id',how='left')
merge4 = pd.merge(merge3,t4,on='id',how='left')
t = merge4
t.shape

(35183, 16)

In [299]:
#将'Trx_Cod1_Cd','Trx_Cod2_Cd','trx_month'转化为类型特征，否则下面统计会报错
df_trd[['Trx_Cod1_Cd','Trx_Cod2_Cd','trx_month']] = df_trd[['Trx_Cod1_Cd','Trx_Cod2_Cd','trx_month']].astype('str')

#交易方向，交易方式，收支一级分类代码，交易月份做细致的分类统计
cols = ['Dat_Flg1_Cd','Dat_Flg3_Cd','Trx_Cod1_Cd','trx_month']
for col in cols:
    for fea in list(df_trd[col].unique()): 
        t1 = df_trd[df_trd[col]==fea].groupby('id',as_index=False)['cny_trx_amt'].agg(
            {col+'_'+fea+'_sum':'sum',col+'_'+fea+'_count':'count',col+'_'+fea+'_min':'min',
             col+'_'+fea+'_max':'max',col+'_'+fea+'_std':'std',col+'_'+fea+'_mean':'mean'})
        t2 = df_trd[df_trd[col]==fea].groupby('id',as_index=False)['trx_day'].agg(
            {col+'_'+fea+'_days':'nunique'})
        merge1 = pd.merge(t,t1,on='id',how='left')
        merge2 = pd.merge(merge1,t2,on='id',how='left')
        t = merge2
    print(col+' is done')

#收支二级分类代码特征取值太多，只计算分类交易次数
col = 'Trx_Cod2_Cd'
for fea in list(df_trd[col].unique()): 
    t1 = df_trd[df_trd[col]==fea].groupby('id',as_index=False)['cny_trx_amt'].agg({col+'_'+fea+'_count':'count'})
    t = pd.merge(t,t1,on='id',how='left')
print(col+' is done')
t.shape

Dat_Flg1_Cd is done
Dat_Flg3_Cd is done
Trx_Cod1_Cd is done
trx_month is done
Trx_Cod2_Cd is done


(35183, 143)

In [300]:
t['day_trx_amt'] = t['trx_sum']/t['trx_days']#每日交易金额
t['one_trx_amt'] = t['trx_count']/t['trx_days']#每日交易次数

t['total_trx_amt'] = t['Dat_Flg1_Cd_C_sum'] - t['Dat_Flg1_Cd_B_sum']#交易额总和
t['income_ratio'] = t['Dat_Flg1_Cd_C_sum']/t['total_trx_amt']#收入占交易总和比重
t['expend_ratio'] = -t['Dat_Flg1_Cd_B_sum']/t['total_trx_amt']#支出占交易总和比重

t['day_income_amt'] = t['Dat_Flg1_Cd_C_sum']/t['Dat_Flg1_Cd_C_days']#单日收入金额
t['day_expend_amt'] = -t['Dat_Flg1_Cd_B_sum']/t['Dat_Flg1_Cd_B_days']#单日支出金额

df_trd1 = t.copy()
df_trd1.shape

trd is done


In [302]:
#加载App访问数据
train_beh = pd.read_csv('train_data/train_beh.csv')
test_beh = pd.read_csv('test_data/test_beh.csv')

In [303]:
#将训练数据和测试数据连接方便做特征
df_beh = pd.concat([train_beh,test_beh],axis=0,ignore_index=True)
df_beh['flag'] = df_beh['flag'].fillna(-1)

In [304]:
df_beh['visit_tm'] = pd.to_datetime(df_beh['Unnamed: 3'])
df_beh['visit_day'] = df_beh['visit_tm'].dt.day#提取访问日期

df_beh['last_visit_day'] = df_beh.groupby('id')['visit_day'].transform('max')#最后访问日期
df_beh['last_visit_tm'] = df_beh.groupby('id')['visit_tm'].transform('max')#最后访问时间
df_beh.head()

id  flag page_no           Unnamed: 3  page_tm Unnamed: 2  \
0  U64F0C9   0.0     SZA  2019-06-30 12:44:27      NaN        NaN   
1  U64F0C9   0.0     CQE  2019-06-25 16:15:11      NaN        NaN   
2  U64F0C9   0.0     AAO  2019-06-30 12:44:17      NaN        NaN   
3  U64F0C9   0.0     CQE  2019-06-17 13:50:12      NaN        NaN   
4  U64F0C9   0.0     AAO  2019-06-17 13:50:08      NaN        NaN   

             visit_tm  visit_day  last_visit_day       last_visit_tm  
0 2019-06-30 12:44:27       30.0            30.0 2019-06-30 12:44:27  
1 2019-06-25 16:15:11       25.0            30.0 2019-06-30 12:44:27  
2 2019-06-30 12:44:17       30.0            30.0 2019-06-30 12:44:27  
3 2019-06-17 13:50:12       17.0            30.0 2019-06-30 12:44:27  
4 2019-06-17 13:50:08       17.0            30.0 2019-06-30 12:44:27

In [305]:
t0 = df_beh[['id','flag']].drop_duplicates().reset_index(drop=True)
t1 = df_beh.groupby('id',as_index=False)['page_no'].agg({'page_count':'count','page_unique':'nunique'})#统计访问页面次数，访问页面数量
t2 = df_beh[df_beh['visit_day']==df_beh['last_visit_day']].groupby('id',as_index=False)['page_no'].agg(
    {'last_page_cnt':'count','last_page_unique':'nunique'})#统计最后一天访问页面次数，访问页面数量
t3 = df_beh.groupby('id',as_index=False)['visit_day'].agg({'day_unique':'nunique'})#访问天数
merge1 = pd.merge(t0,t1,on='id',how='left')
merge2 = pd.merge(merge1,t2,on='id',how='left')
merge3 = pd.merge(merge2,t3,on='id',how='left')
t = merge3

#各访问页面的访问次数分类统计
col = 'page_no'
for fea in list(df_beh[col].unique()): 
    t1 = df_beh[df_beh[col]==fea].groupby('id',as_index=False)['page_no'].agg({col+'_'+fea+'_count':'count'})
    t = pd.merge(t,t1,on='id',how='left')
print(col+' is done')
df_beh1 = t.copy()
df_beh1.shape

page_no is done


(13145, 36)

In [313]:
#将三表连接，并填充缺失值
temp = pd.merge(df_tag1,df_trd1,on=['id','flag'],how='left')
df = pd.merge(temp,df_beh1,on=['id','flag'],how='left')
df = df.fillna(0)

#将训练数据和测试数据分开
df_train = df[df['flag']!=-1]
df_test = df[df['flag']==-1].drop(['flag'],axis=1)

In [319]:
#将数据导出
df_train.to_csv('df_train.csv',index=None)
df_test.to_csv('df_test.csv',index=None)